# Section 1

## Object Detection through HTTP Request

### Scala TF dependencies and imports

In [ ]:
interp.load.ivy(coursierapi.Dependency.of("org.platanios", "tensorflow_2.12", "0.4.1").withClassifier("linux-cpu-x86_64"))
interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")

In [ ]:
import org.platanios.tensorflow.api._
import org.platanios.tensorflow.api.learn._
import org.platanios.tensorflow.api.learn.layers._
import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator
import org.platanios.tensorflow.data.image.MNISTLoader
import org.platanios.tensorflow.api.core.client.FeedMap
import org.tensorflow.framework.GraphDef

import org.platanios.tensorflow.api.ops.Files
import org.platanios.tensorflow.api.ops.Image
import scala.io.Source

In [ ]:
import java.net.URL
import sys.process._
import java.io.{BufferedInputStream, File, FileInputStream}

## Prepare the model to be served

### We download the published Tensorflow model from a public URL

See https://github.com/tensorflow/models for reference on available models from research community.

In [ ]:
val cacheDir = "resources"//sys.env("HOME") + "/data/models/tmp"
// val modelName = "ssd_mobilenet_v2_coco_2018_03_29"
val modelName = "mask_rcnn_resnet50_atrous_coco_2018_01_28"
val archiveFilename = s"${modelName}.tar.gz"

// val modelURL = s"http://download.tensorflow.org/models/object_detection/${archiveFilename}"
val modelURL = s"http://download.tensorflow.org/models/object_detection/$archiveFilename"

// THIS CELL WAS CHANGED TO MARKDOW TO AVOID EXECUTING A HEAVY DOWNLOAD THAT WAS ALREADY DONE

new URL(modelURL) #> new File(s"${cacheDir}/${archiveFilename}") !

### Extraction of the model archive

// THIS CELL WAS CHANGED TO MARKDOW TO AVOID EXECUTING AN UNARCHIVE THAT WAS ALREADY DONE
s"tar -xzf ${cacheDir}/${archiveFilename} -C ${cacheDir}" !

In [ ]:
s"ls ${cacheDir}/${modelName}"!

### Load the model in a Tensorflow Session

In the extracted directory, named after the `modelName`, we are interested in loading the `frozen_inference_graph.pb`.

In [ ]:
val modelGraphPath = s"${cacheDir}/${modelName}_2018_01_28/frozen_inference_graph.pb"

In [ ]:
lazy val graphDef = GraphDef.parseFrom(
    new BufferedInputStream(new FileInputStream(new File(modelGraphPath))))
val graph = Graph.fromGraphDef(graphDef)
val session = Session(graph)

### Add nodes to access the model signature (input and responses)

`image_tensor` is the tensor representing the input images, with 3-channels colors

In [ ]:
val imagePlaceholder = graph.getOutputByName("image_tensor:0").toUByte
imagePlaceholder.shape

`num_detections` is the number of detected objects

`detection_boxes` are the corrdinates of detected objects

`detection_scores` are a probability-like score for each object

`detection_classes` are the class label for each detection

**TODO: Complete the definition of Outputs**

In [ ]:
val detectionBoxes = graph.getOutputByName("detection_boxes:0")
val detectionScores = graph.getOutputByName("detection_scores:0")
val detectionClasses = graph.getOutputByName("detection_classes:0")
val numDetections = graph.getOutputByName("num_detections:0")

### Nodes to feed the graph with an image file path

A Placeholder to provide a filepathe is the input, and the file is opened and decoded as an image:

In [ ]:
val (imgTensor, fileNamePlaceholder) = tf.createWith(graph = graph) {
    val fileNamePlaceholder = tf.placeholder[String]()
    val fileTensor = Files.readFile(fileNamePlaceholder)
    val imgTensor = Image.decodePng(fileTensor, 3)
    (imgTensor, fileNamePlaceholder)
  }

## Define the Service function


In [ ]:
def detectObjects(file: File) = {
    
    // Feed the image file to get the Images Tensor
    val fileNameTensor = Tensor.fill(Shape())(file.getAbsolutePath())
    val feedImg = FeedMap(fileNamePlaceholder, fileNameTensor)
    val imageOuts: Tensor[UByte] =
      session.run(fetches = imgTensor, feeds = feedImg)

    // Retain image sizes to format output later
    val width = imageOuts.shape(1)
    val height = imageOuts.shape(0)

    // Feed with Images to compute detections:
    val feeds = FeedMap(imagePlaceholder, imageOuts.slice(NewAxis, ---))
    val Seq(boxes, scores, classes, num) =
      session.run(
        fetches =
          Seq(detectionBoxes, detectionScores, detectionClasses, numDetections),
        feeds = feeds)
    
  val labelList =
      for {
        i <- 0 until num(0).scalar.asInstanceOf[Float].toInt
        labelId = classes(0, i).toFloat.scalar.toInt
        //label = labelMap.getOrElse(labelId, "unknown")
        //if setOfClasses.isEmpty || setOfClasses.contains(label)

        box = boxes(0, i).toFloat.entriesIterator.toSeq
        x1 = (box(1) * width).toInt
        y1 = (box(0) * height).toInt
        x2 = (box(3) * width).toInt
        y2 = (box(2) * height).toInt
        labelBox = (x1, y1, x2 - x1 + 1, y2 - y1 + 1)
        score = scores(0, i).toFloat.scalar
      } yield (labelId, score, x1, y1, x2, y2)
    labelList.toSeq
}

In [ ]:
detectObjects(new File("resources/baywatch.jpg"))

## Transform Labels IDs in labels

Link number label to its text representation

In [ ]:
val labelsMapPath = "resources/coco_labels.txt"

In [ ]:
val labelsMapToString = Source.fromFile(labelsMapPath).getLines.map { line =>
        val splitLine = line.split(" ")
        splitLine(0).toInt -> splitLine(1)
    }.toMap

In [ ]:
import $ivy.`io.circe:circe-core_2.12:0.10.1`
import $ivy.`io.circe:circe-generic_2.12:0.10.1`
import $ivy.`io.circe:circe-parser_2.12:0.10.1`
import _root_.io.circe.{Decoder, Encoder}

case class Detection(label: String, score: Float, x1: Int, y1: Int, x2: Int, y2: Int)

object Detection {
  import _root_.io.circe.generic.semiauto._

  implicit lazy val encoder: Encoder[Detection] = deriveEncoder[Detection]
  implicit lazy val decoder: Decoder[Detection] = deriveDecoder[Detection]
}

In [ ]:
def labelize(detections: Seq[(Int, Float, Int, Int, Int, Int)]) = detections.map { 
    d => d match {
        case (id, score, x1, y1, x2, y2) if (labelsMapToString.contains(id)) => Detection(labelsMapToString(id), score, x1, y1, x2, y2)
        case (id, score, x1, y1, x2, y2) => Detection("unknown", score, x1, y1, x2, y2)
    }
}

In [ ]:
labelize(detectObjects(new File("resources/baywatch.jpg")))

## Run an akka-http service

In [ ]:
import $ivy.`com.typesafe.akka:akka-http_2.12:10.1.5`
import $ivy.`com.typesafe.akka:akka-actor_2.12:2.5.18`
import $ivy.`com.typesafe.akka:akka-stream_2.12:2.5.18`
import $ivy.`de.heikoseeberger:akka-http-circe_2.12:1.21.0`

import akka.actor.ActorSystem
import akka.stream.{IOResult, Materializer}
import akka.stream.scaladsl.{FileIO, Source}
import akka.http.scaladsl.Http
import akka.http.scaladsl.Http.ServerBinding
import akka.http.scaladsl.server.Directives._
import akka.http.scaladsl.server.Route
import akka.stream.ActorMaterializer
import akka.util.ByteString

import scala.util.{Failure, Success}
import java.nio.file.{Files, Paths}

In [ ]:
implicit val system = ActorSystem("main-system")
implicit val materializer = ActorMaterializer()
implicit val executionContext = system.dispatcher

import de.heikoseeberger.akkahttpcirce.FailFastCirceSupport._

val route =
  path("detect") {
      post {
          withoutRequestTimeout {
              fileUpload("img") {
                  case (metadata, byteSource) =>
                  val file = File.createTempFile("image", ".png")
                  val fileSaveFut = byteSource.runWith(FileIO.toPath(Paths.get(file.getAbsolutePath)))
                  onComplete(fileSaveFut) {
                      case Success(s) => 
                          val detections = labelize( detectObjects(file))
                          complete(detections)
                      case Failure(s) => complete(s.getMessage)
                  }
              }
          }
      }
  }



In [ ]:
val bindingFuture = Http().bindAndHandle(route, "localhost", 8080)

// DO NOT EXECUTE UNTIL YOU WANT TO KILL THE SERVER!

bindingFuture
      .flatMap(_.unbind()) // trigger unbinding from the port
      .onComplete(_ => system.terminate()) // and shutdown when done
